<a href="https://colab.research.google.com/github/Jed-77/tensorflow-deeplearning/blob/master/TF2_0_Stock_Return_Prediction_Pt_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import tensorflow as tf
from tensorflow.keras.layers import Input, SimpleRNN, GRU, LSTM, Flatten, Dense, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [0]:
# This model we will output prediction of up or down, rather than regression
# ... of next day return. 

# Get the stock data
df = pd.read_csv('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/tf2.0/sbux.csv')
df['return'] = (df['close']-df['close'].shift(1))/df['close']
input_data = df[['open', 'high', 'low', 'close', 'volume']].values
targets = df['return'].values

# Make the data for the neural network
T = 10  # still considering time intervals of 10, predicting the 11th
D = input_data.shape[1]   # using all 5 features (open, high, low, close, vol)
N = len(input_data) - T  # the number of sampes 

# Normalise the inputs of training data
Ntrain = len(input_data)*2//3
scaler = StandardScaler()
scaler.fit(input_data[:Ntrain+T])
input_data = scaler.transform(input_data)

In [0]:
# Prepare training data
X_train = np.zeros((Ntrain, T, D))
Y_train = np.zeros(Ntrain)
for t in range(Ntrain):
  X_train[t, :, :] = input_data[t:t+T]
  Y_train[t] = (targets[t+T] > 0)

# Prepare test  data
X_test = np.zeros((N - Ntrain, T, D))
Y_test = np.zeros(N - Ntrain)
for u in range(N- Ntrain):
  # u counts from 0 -> N-Ntrain, t counts from Ntrain->N
  t = u+Ntrain
  X_test[u, :, :] = input_data[t:t+T]
  Y_test[u] = (targets[t+T]>0)


In [0]:
# Build A Model
i = Input(shape=(T,D))
x = LSTM(50)(i)
x = Dense(1, activation='sigmoid')(x)
model = Model(i, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(lr=0.01),
    metrics=['accuracy']
)

# Train the model
r = model.fit(
    X_train, Y_train,
    epochs=80,
    validation_data=(X_test, Y_test)
)

Epoch 1/80
27/27 [==============================] - 1s 24ms/step - loss: 0.7006 - accuracy: 0.5101 - val_loss: 0.6961 - val_accuracy: 0.4902
Epoch 2/80
27/27 [==============================] - 0s 7ms/step - loss: 0.6957 - accuracy: 0.5113 - val_loss: 0.6938 - val_accuracy: 0.4927
Epoch 3/80
27/27 [==============================] - 0s 7ms/step - loss: 0.6915 - accuracy: 0.4970 - val_loss: 0.6928 - val_accuracy: 0.4829
Epoch 4/80
27/27 [==============================] - 0s 8ms/step - loss: 0.6919 - accuracy: 0.5221 - val_loss: 0.6911 - val_accuracy: 0.5317
Epoch 5/80
27/27 [==============================] - 0s 8ms/step - loss: 0.6901 - accuracy: 0.5411 - val_loss: 0.6909 - val_accuracy: 0.5293
Epoch 6/80
27/27 [==============================] - 0s 7ms/step - loss: 0.6899 - accuracy: 0.5328 - val_loss: 0.6955 - val_accuracy: 0.4878
Epoch 7/80
27/27 [==============================] - 0s 7ms/step - loss: 0.6913 - accuracy: 0.5328 - val_loss: 0.6913 - val_accuracy: 0.5293
Epoch 8/80
27/27 [=